In [72]:
import pandas as pd
import numpy as np
from pathlib import Path

In [73]:
# hugoferq
work =  Path(r'D:\\Trabajo\AITeacherAllocation')

In [74]:
# Mariadelaguarda
work =  Path(r'D:\\Trabajo\AITeacherAllocation')

In [75]:
df = pd.read_csv(work/r'Results\\Base consolidada.csv')

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_2756\3792860488.py:1: DtypeWarning: Columns (1,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(work/r'Results\\Base consolidada.csv')


# Target population

The goal is predict teacher allocation in this population target:

- Educational stage
    - Kindergarten 
    - Primary school

- Teachers:
    - Teachers
    - Education Assistant

- Type of school 
    - Public managed by Central Goverment

In [76]:
# Relevant rows
df = df.loc[df['niv_mod']!='F0'] # Educational stage
df = df.dropna(subset=['region']) # Drop missing in region
df = df.loc[(df['gestion']==1) & (df['ges_dep']=='A1')] # Type of school
df = df.loc[df['estado']==1] # Active in 2022

In [77]:
# Repair educational stage
df.loc[df['d_niv_mod']=='Inicial - Jard¡n','d_niv_mod'] = 'Inicial - Jardin'
df.loc[df['d_niv_mod']=='Inicial - Cuna-jard¡n','d_niv_mod'] = 'Inicial - Cuna-jardin'
# Drop few observations 
df = df.loc[df['d_niv_mod']!='Inicial - Cuna'] # 39 rows deleted
# Drop no relevant information in target variable
df = df.loc[df['secciones_necesarias']!=0] #720 rows deleted

In [78]:
# Drop irrelant columns
df = df.drop(columns=['Unnamed: 0.1','usuario_minedu','bolsa_nexus','bolsa_sira',
'dir_req', 'sub_dir_req','jer_req','fisica_req','aip_req','dir_exd','sub_dir_exd',
'jer_exd','niv_mod','gestion','d_gestion','ges_dep','d_ges_dep','estado',
'd_estado','d_region','tipo_entidad','jec','Unnamed: 0','year','otro_doc_exd','d_dpto','codlocal','d_dist'])

In [79]:
# Other outcomes not relevant to predict in this version
other_outcomes=['doc_aula_req','aux_req','doc_aula_exd_n', 'doc_aula_exd_c', 'aux_exd']
df = df.drop(columns=other_outcomes)

# Feature storage 

## Numbers variables

In [96]:
workers = ['dir_n','dir_c','sub_dir_n','sub_dir_c', 'doc_aula_n', 'doc_aula_c', 'otro_doc_n', 'otro_doc_c', 'aux_n', 'aux_c']

In [81]:
enrollment = [x for x in df.columns.to_list() if (x.find('cant')!=-1) or (x.find('inclu')!=-1)]

In [82]:
gps = ['nlat_ie','nlong_ie','ubigeo','codooii']

In [83]:
numerical_inputs = workers+enrollment+gps

In [ ]:
# Revisar los missing y completar con 0

In [97]:
workers_stats = df[workers].describe().transpose()

In [98]:
workers_stats['cv'] = workers_stats.iloc[:,2] / workers_stats.iloc[:,1] * 100
workers_stats

,count,mean,std,min,25%,50%,75%,max,cv
dir_n,160087.0,0.131222,0.337644,0.0,0.0,0.0,0.0,1.0,257.307046
dir_c,160087.0,0.049904,0.217748,0.0,0.0,0.0,0.0,1.0,436.331832
sub_dir_n,160087.0,0.022769,0.151782,0.0,0.0,0.0,0.0,2.0,666.619100
sub_dir_c,160087.0,0.013187,0.131522,0.0,0.0,0.0,0.0,5.0,997.390957
doc_aula_n,160087.0,2.332925,4.473909,0.0,0.0,1.0,2.0,66.0,191.772494
doc_aula_c,160087.0,1.055745,1.484930,0.0,0.0,1.0,1.0,27.0,140.652410
otro_doc_n,160087.0,0.058543,0.342958,0.0,0.0,0.0,0.0,7.0,585.820524
otro_doc_c,160087.0,0.125319,0.478971,0.0,0.0,0.0,0.0,8.0,382.200324
aux_n,160087.0,0.063291,0.404233,0.0,0.0,0.0,0.0,12.0,638.693678
aux_c,160087.0,0.150443,0.594403,0.0,0.0,0.0,0.0,15.0,395.101118


## Categorical variables

In [84]:
categorical=['d_niv_mod', 'ruralidad', 'vraem', 'frontera', 'bilingue',
 'caracteristica','d_prov','d_dreugel']

# Train/Test split

In [86]:
X = df[numerical_inputs]
y = df[['secciones_necesarias']]

In [89]:
X

,dir_n,dir_c,sub_dir_n,sub_dir_c,jer_n,jer_c,doc_aula_n,doc_aula_c,otro_doc_n,otro_doc_c,...,inclu1 (t-1),inclu2 (t-1),inclu3 (t-1),inclu4 (t-1),inclu5 (t-1),inclu6 (t-1),nlat_ie,nlong_ie,ubigeo,codooii
1,0,0,0,0,0,0,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-14.18550,-72.78550,30701.0,30007.0
2,0,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-14.16780,-72.79500,30701.0,30007.0
3,0,0,0,0,0,0,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,-14.14340,-72.73650,30701.0,30007.0
4,0,0,0,0,0,0,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-14.12080,-72.73180,30701.0,30007.0
5,0,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,-14.19309,-72.79635,30701.0,30007.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192719,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,-11.85750,-75.42010,120413.0,120005.0
192722,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,-11.92314,-70.09361,170103.0,170001.0
192723,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,-12.83880,-74.07473,50410.0,50004.0
192724,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,-13.61076,-74.42475,51011.0,50010.0


In [60]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.1, random_state=0)